In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.stattools import adfuller

import pickle
import joblib


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
path = r"D:\A\miniproject3\datasets\final_dataset.csv"

if os.path.exists(path):
    final_df = pd.read_csv(path)
    print("Dataset Loaded Successfully")
else:
    print("Error: File not found!")


In [ ]:
train_size = int(len(final_df) * 0.8)
train, test = final_df[:train_size], final_df[train_size:]


In [ ]:

def adf_test(series):
    result = adfuller(series)
    print("ADF Statistic:", result[0])
    print("p-value:", result[1])
    if result[1] < 0.05:
        print("Data is stationary")
    else:
        print("Data is not stationary")

adf_test(final_df["Price"]) 


In [ ]:
final_df["Price_Diff"] = final_df["Price"].diff().dropna()
adf_test(final_df["Price_Diff"].dropna())

### Step 5: ACF & PACF Plots
plt.figure(figsize=(12,5))
plot_acf(final_df["Price"].dropna(), lags=40)
plot_pacf(final_df["Price"].dropna(), lags=40)
plt.show()


In [ ]:
auto_model = auto_arima(final_df["Price"], seasonal=False, trace=True)
print(auto_model.summary())


In [ ]:
p, d, q = auto_model.order  
model = ARIMA(final_df["Price"], order=(p, d, q))
model_fit = model.fit()
print(model_fit.summary())


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(train["Price"], order=(p,d,q)) 
model_fit = model.fit()

forecast = model_fit.forecast(steps=len(test))


In [ ]:

mae = mean_absolute_error(test["Price"], forecast)
mse = mean_squared_error(test["Price"], forecast)
print(f"MAE: {mae}, MSE: {mse}")


In [ ]:
# with open("arima_model.pkl", "wb") as model_file:
#     pickle.dump(model_fit, model_file)

# with open("arima_model.pkl", "rb") as model_file:
#     loaded_model = pickle.load(model_file)

# forecast = loaded_model.forecast(steps=30)
